In [ ]:
!wget -O OD_SS.zip https://arquivos.ufsc.br/f/981acdaa6d2442f68c1a/?dl=1

In [ ]:
!unzip OD_SS.zip

In [ ]:
!mkdir FeulgenDataset
!mkdir FeulgenDataset/images
!mkdir FeulgenDataset/images/train
!mkdir FeulgenDataset/images/test
!mkdir FeulgenDataset/images/val
!mkdir FeulgenDataset/labels

In [ ]:
!cp -r /content/OD_SS/train/Images/* /content/FeulgenDataset/images/train
!cp -r /content/OD_SS/test/Images/* /content/FeulgenDataset/images/test
!cp -r /content/OD_SS/val/Images/* /content/FeulgenDataset/images/val
!cp -r /content/OD_SS/train/SS_Labels/* /content/FeulgenDataset/labels
!cp -r /content/OD_SS/test/SS_Labels/* /content/FeulgenDataset/labels
!cp -r /content/OD_SS/val/SS_Labels/* /content/FeulgenDataset/labels

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!/opt/bin/nvidia-smi
!nvcc --version

In [ ]:
!pip install fastai==2.3.0
!pip install lapixdl==0.7.15
!pip install -U albumentations

In [ ]:
from pathlib import Path
import fnmatch
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt


from fastai.vision.all import *

import albumentations as A
import cv2

from lapixdl.evaluation.visualize import show_segmentations
from lapixdl.evaluation.model import Result

In [ ]:
from fastai.vision.models import resnet18
used_model = resnet18

In [ ]:
path = Path('/content')

In [ ]:
!mkdir Output

In [ ]:
path_dataset = path/'FeulgenDataset'
path_lbl = path_dataset/'labels'
path_img = path_dataset/'images'
path_models = Path("/content/Output")
lbl_names = get_image_files(path_lbl)
len(lbl_names)

In [ ]:
fnames = get_image_files(path_img, folders=['train', 'val'])

get_mask = lambda x: path_lbl/f'{x.stem}{x.suffix}'
mask = PILMask.create(get_mask(fnames[1]))
src_size = np.array(mask.shape)
src_size

In [ ]:
config_file_content = '''background
alterada
intermediaria
velhas
sujeira
sobreposicao
nao_identificado
neutrofilo
mancha'''

codes_save_path = "/content/codes.txt"

f = open(codes_save_path, "w")
f.write(config_file_content)
f.close()

In [ ]:
codes = np.loadtxt(codes_save_path, dtype=str) 
codes[0] = 'background' # To show in the visualization
codes

# Data Analisys

In [ ]:
img_f = fnames[random.randint(0, len(fnames) - 1)]

img = PILImage.create(img_f)
mask = PILMask.create(get_mask(img_f))

fig, axes = show_segmentations([Result(np.array(img), np.array(mask))], codes)

# Metrics, augmentations, data split and model definition 

In [ ]:
# Fix seed
import imgaug
import random
random.seed(81615)
imgaug.seed(81615)

class SegmentationAlbumentationsTransform(ItemTransform):
    split_idx = 0 #Train only
    order = 2 #After resize
    def __init__(self, aug): self.aug = aug
    def encodes(self, x):
        img,mask = x
        aug = self.aug(image=np.array(img), mask=np.array(mask))
        return PILImage.create(aug["image"]), PILMask.create(aug["mask"])

class ImageResizer(Transform):
    order=1
    "Resize image to `size` using `resample`"
    def __init__(self, size, resample=Image.BILINEAR):
        if not is_listy(size): size=(size,size)
        self.size,self.resample = (size[1],size[0]),resample

    def encodes(self, o:PILImage): return o.resize(size=self.size, resample=self.resample)
    def encodes(self, o:PILMask):  return o.resize(size=self.size, resample=Image.NEAREST)

zoom_augmentation = lambda img_shape: A.Compose([
    A.RandomScale(scale_limit=(0, 0.1), p=0.75),
    A.CenterCrop(img_shape[0], img_shape[1])
])

augmentations = lambda img_shape: A.Compose([
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Rotate((-10, 10), p=0.75),
    zoom_augmentation(img_shape),
    # A.RandomBrightnessContrast(0.1, 0.1, p=0.75),
    A.Affine(p=0.75, shear=0.2)
])

In [ ]:
tfms = [[PILImage.create], [get_mask, PILMask.create, AddMaskCodes(codes)]]
folder_split = FuncSplitter(lambda fname: Path(fname).parent.name == 'val')
src = Datasets(fnames, tfms, splits=folder_split(fnames))

In [ ]:
class DiceMulti(Metric):
    "Averaged Dice metric (Macro F1) for multiclass target in segmentation"
    def __init__(self, axis=1): self.axis = axis
    def reset(self): self.inter,self.union,self.total_area = {},{},{}
    def accumulate(self, learn):
        pred,targ = flatten_check(learn.pred.argmax(dim=self.axis), learn.y)
        for c in range(learn.pred.shape[self.axis]):
            p = torch.where(pred == c, 1, 0)
            t = torch.where(targ == c, 1, 0)
            c_inter = (p*t).float().sum().item()
            c_totalSumAreas = (p+t).float().sum().item()
            c_union = c_totalSumAreas-c_inter
            if c in self.inter:
                self.inter[c] += c_inter
                self.total_area[c] += c_totalSumAreas
                self.union[c] += c_union
            else:
                self.inter[c] = c_inter
                self.total_area[c] = c_totalSumAreas
                self.union[c] = c_union
    @property
    def value(self):
        binary_dice_scores = np.array([])
        for c in self.inter:
            binary_dice_scores = np.append(binary_dice_scores, 2.*self.inter[c]/self.total_area[c] if self.total_area[c] > 0 else np.nan)
        return np.nanmean(binary_dice_scores)

class JaccardCoeffMulti(DiceMulti):
  "Implementation of the Averaged Jaccard coefficient that is lighter in RAM -- Mean IoU (Intersection Over Union)"
  @property
  def value(self): 
    binary_jaccard_scores = np.array([])
    for c in self.inter:
        binary_jaccard_scores = np.append(binary_jaccard_scores, self.inter[c]/self.union[c] if self.union[c] > 0 else np.nan)
    return np.nanmean(binary_jaccard_scores)

def acc_metric(ipt, target):
  return foreground_acc(ipt, target, bkg_idx=0)

iou_metric = JaccardCoeffMulti()

f1_metric = DiceMulti()

metrics = [acc_metric, iou_metric, f1_metric]

In [ ]:
def get_results(predict, size, n=3):
  for fname in random.sample(fnames, n):
    res = predict(fname)
    yield Result(
        np.array(PILImage.create(fname).resize((size[1], size[0]))), 
        np.array(PILMask.create(get_mask(fname)).resize((size[1], size[0]))),
        np.array(res[0])
    )

# Training

In [ ]:
def get_learner(data, load_model=None, unfreeze:bool=False):
  learn = unet_learner(data,
                       used_model, 
                       metrics=metrics, 
                       # loss_func=CrossEntropyLossFlat(axis=1)
                       )
  learn.path = path_models
  if(load_model != None):
    learn.load(load_model, with_opt=True)
  if(unfreeze):
    learn.unfreeze()
  return learn

### 300 x 400

In [ ]:
size = src_size//4
bs = 16
transforms = [
  ImageResizer((size[0], size[1])), 
  ToTensor(), 
  IntToFloatTensor(),
  SegmentationAlbumentationsTransform(augmentations(size)) 
]
print(size)

In [ ]:
data = src.dataloaders(bs=bs, after_item=transforms)

In [ ]:
data.train.show_batch(max_n=6)

In [ ]:
get_learner(data).lr_find()

In [ ]:
lr=slice(1e-3)
wd=1e-3
learn = get_learner(data)
callback = SaveModelCallback(monitor='dice_multi', fname='best_model_300x400_stg1', with_opt=True)

learn.fit_one_cycle(15, lr_max=lr, wd=wd, cbs=callback)

In [ ]:
learn.show_results(max_n=6, figsize=(20,30))

In [ ]:
get_learner(data, 'best_model_300x400_stg1', True).lr_find()

In [ ]:
lr = slice(1e-4,1e-3)
wd=1e-3
learn = get_learner(data, 'best_model_300x400_stg1', True)
callback = SaveModelCallback(monitor='dice_multi', fname='best_model_300x400_stg2', with_opt=True)

learn.fit_one_cycle(10, lr_max=lr, wd=wd, cbs=callback)

In [ ]:
learn = get_learner(data, 'best_model_300x400_stg2')

In [ ]:
fig, axes = show_segmentations(list(get_results(learn.predict, size)), codes, mask_alpha=0.5)

### 600 x 800

In [ ]:
size = src_size//2
bs = 4
transforms = [
  ImageResizer((size[0], size[1])), 
  ToTensor(), 
  IntToFloatTensor(),
  SegmentationAlbumentationsTransform(augmentations(size)) 
]
print(size)

In [ ]:
data = src.dataloaders(bs=bs, after_item=transforms)

In [ ]:
get_learner(data, 'best_model_300x400_stg2').lr_find()

In [ ]:
lr=slice(1e-4)
wd=1e-3
learn = get_learner(data)
callback = SaveModelCallback(monitor='dice_multi', fname='best_model_600x800_stg1', with_opt=True)

learn.fit_one_cycle(15, lr_max=lr, wd=wd, cbs=callback)

In [ ]:
get_learner(data, 'best_model_600x800_stg1', True).lr_find()

In [ ]:
lr = slice(1e-4,1e-3)
wd=1e-3
learn = get_learner(data, 'best_model_600x800_stg1', True)
callback = SaveModelCallback(monitor='dice_multi', fname='best_model_600x800_stg2', with_opt=True)

learn.fit_one_cycle(10, lr_max=lr, wd=wd, cbs=callback)

In [ ]:
fig, axes = show_segmentations(list(get_results(learn.predict, size)), codes, mask_alpha=0.5)

### 1200 x 1600

In [ ]:
size = src_size
bs = 1
transforms = [
  ImageResizer((size[0], size[1])), 
  ToTensor(), 
  IntToFloatTensor(),
  SegmentationAlbumentationsTransform(augmentations(size)) 
]
print(size)

In [ ]:
data = src.dataloaders(bs=bs, after_item=transforms)

In [ ]:
get_learner(data, 'best_model_600x800_stg2').lr_find()

In [ ]:
lr=slice(1e-3)
wd=1e-3
learn = get_learner(data, 'best_model_600x800_stg2')
callback = SaveModelCallback(monitor='dice_multi', fname='best_model_1200x1600_stg1', with_opt=True)

learn.fit_one_cycle(15, lr_max=lr, wd=wd, cbs=callback)

In [ ]:
get_learner(data, 'best_model_1200x1600_stg1', True).lr_find()

In [ ]:
# lr = slice(1e-7,1e-2)
lr = slice(1e-3)
wd=1e-3
learn = get_learner(data, 'best_model_1200x1600_stg1', True)
callback = SaveModelCallback(monitor='dice_multi', fname='best_model_1200x1600_stg2', with_opt=True)

learn.fit_one_cycle(10, lr_max=lr, wd=wd, cbs=callback)

In [ ]:
fig, axes = show_segmentations(list(get_results(learn.predict, size)), codes, mask_alpha=0.5)

# Evaluation

In [ ]:
from lapixdl.evaluation.evaluate import evaluate_segmentation

def gt_mask_iterator_from_image_files(fnames, size):
  for fname in fnames:
    yield np.array(PILMask.create(get_mask(fname)).resize((size[1], size[0])))

def pred_mask_iterator_from_image_files(fnames, size, predict):
  for fname in fnames:
    res = predict(fname)
    yield np.array(res[0])

In [ ]:
size = (1200, 1600)

test_image_files = get_image_files(path_img/'test')
transforms = [
  ImageResizer((size[0], size[1])),
  ToTensor(), 
  IntToFloatTensor()
]

tfms = [[PILImage.create], [get_mask, PILMask.create, AddMaskCodes(codes)]]
src = Datasets(test_image_files, tfms)
test_dl = src.dataloaders(bs=1, after_item=transforms)

learn = get_learner(test_dl, 'best_model_1200x1600_stg2')

gt_masks = gt_mask_iterator_from_image_files(test_image_files, size)
pred_masks = pred_mask_iterator_from_image_files(test_image_files, size, learn.predict)

In [ ]:
eval = evaluate_segmentation(gt_masks, pred_masks, codes)
eval.show_confusion_matrix()

In [ ]:
interp = SegmentationInterpretation.from_learner(learn, dl=test_dl[0])

In [ ]:
interp.plot_top_losses(10)